In [1]:
import io
import os
import time
import pickle
from pymongo import MongoClient

# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud import storage

In [2]:
#connect to database
mongo_client = MongoClient()
speech2text = mongo_client.speech2text

In [3]:
videos = speech2text.videos

In [5]:
# Instantiates a client
speech_client = speech.Client()
storage_client = storage.Client(project='speech-to-nlp')

In [6]:
# connect to bucket. buckets are storage depots
try:
    bucket = storage_client.get_bucket('galen_interviews')
except google.cloud.exceptions.NotFound:
    print('Sorry, that bucket does not exist!')

In [7]:
# create sample instance using whatever audiofile you want to transcribe
audio_sample = speech_client.sample(
            source_uri='gs://galen_interviews/file1.raw',
            encoding='LINEAR16',
            sample_rate=16000)

In [8]:
#this is the "model.fit()" of the speech api
operation = speech_client.speech_api.async_recognize(audio_sample)

retry_count = 3000
while retry_count > 0 and not operation.complete:
    retry_count -= 1
    time.sleep(2)
    operation.poll()

if operation.complete:
    print('Operation complete!')
else:
    print('Failed.')

#alternatives come with the transcribed text and a confidence score 
alternatives = operation.results

#put the text into the database
for alternative in alternatives:
    text = ''
    text_dict = {}
    text+= alternative.transcript
    text_dict['text'] = text
    videos.insert_one(text_dict)

Operation complete!


I put the operation above into a python script and ran it through console. It's important that it's wrapped inside of a function and the entire client is reinitialized - otherwise the API connections will stack up (i.e. try to have more than one connection at once) and it will throw a 502 server error. I tried to fix this, but there is currently no explicit way to close a connection before you open another.

The code below is what I was trying to do before without success.

In [ ]:
#create txt with all URI file paths
%%bash
gsutil ls gs://galen_interviews/ >> gs_paths.txt

In [ ]:
# instantiate all of the sample instances
# then loop over audio_samples to API 
audio_samples = []
with io.open('gs_paths.txt', 'rb') as paths:
    for file_name in paths:
        audio_sample = speech_client.sample(
            source_uri=file_name,
            encoding='LINEAR16',
            sample_rate=16000)
        audio_samples.append(audio_sample)